In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

# QUERY SETTLEMENT

In [2]:
settled = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2024\01. ENERO\SETTLEMENT*.csv', recursive=False)
settled

['C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2024\\01. ENERO\\SETTLEMENT 2024-01-31.csv']

In [3]:
settled_fp1 = pd.concat([pd.read_csv(f).assign(file_name=os.path.basename(f)) for f in settled])
settled_fp1.columns = settled_fp1.columns.str.strip().str.lower() 
settled_fp1['report_date_local'] = pd.to_datetime(settled_fp1['report_date_local'])
settled_fp1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,total_released,total_retained,tipopago,total_due,type,created_date,created_time,updated_date,updated_time,file_name


In [4]:
settled_fp1.shape

(0, 26)

In [5]:
settled_fp1['report_date_local'].unique()

<DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]

In [6]:
settled_fp1['type'].unique()

array([], dtype=object)

# IDENTIFIACION DATA

In [7]:
match_2 = settled_fp1[(settled_fp1['type'] != 'DAILY')]
match_2.sort_values(by=['report_date_local','report_id','tipopago'],ascending=True,inplace=True)
match_2.drop_duplicates(subset='report_id', keep='first', inplace=True)
match_2.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,total_released,total_retained,tipopago,total_due,type,created_date,created_time,updated_date,updated_time,file_name


In [8]:
match_2.shape

(0, 26)

In [9]:
duplicates_data = match_2[match_2.duplicated(['report_id'] , keep=False)]
duplicates_data.sort_values(by=['tipopago'],ascending=True,inplace=True)
duplicates_data.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,total_released,total_retained,tipopago,total_due,type,created_date,created_time,updated_date,updated_time,file_name


In [10]:
match_2.columns

Index(['merchant_id', 'merchant_name', 'user_email', 'report_id',
       'report_date_local', 'total_transactions', 'total_fees_charged',
       'total_taxes', 'applied_fees_benefits', 'applied_taxes_benefits',
       'fee_direct_deposit', 'taxes_direct_deposit', 'total_adjustments',
       'total_adjustment_fees_charged', 'total_adjustment_taxes', 'cobranza',
       'total_released', 'total_retained', 'tipopago', 'total_due', 'type',
       'created_date', 'created_time', 'updated_date', 'updated_time',
       'file_name'],
      dtype='object')

In [11]:
settled_final = match_2[['merchant_id','merchant_name','user_email','report_id','report_date_local','total_transactions'
                         ,'total_fees_charged','total_taxes','applied_fees_benefits','applied_taxes_benefits'
                         ,'fee_direct_deposit','taxes_direct_deposit','total_adjustments','cobranza','total_released'
                         ,'total_retained', 'tipopago', 'total_due','type','created_date','created_time','updated_date'
                         ,'updated_time']]
settled_final.sort_values(by=['updated_time'],inplace=True)
settled_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21400\4039159894.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  settled_final.sort_values(by=['updated_time'],inplace=True)


,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,cobranza,total_released,total_retained,tipopago,total_due,type,created_date,created_time,updated_date,updated_time


In [12]:
settled_final.shape

(0, 23)

In [13]:
path_file = r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2024\01. ENERO\ARCHIVO DISPERSIONES FP\Dispersión-FP 2024-01-31.xlsx'
book = load_workbook(path_file)
writer = pd.ExcelWriter(path_file, engine='openpyxl', mode='a', if_sheet_exists='overlay')
writer.workbook = openpyxl.load_workbook(path_file)
settled_final.to_excel(writer, sheet_name='SETTLEMENT', index=False)
book.save(path_file)
writer.close()